<a href="https://colab.research.google.com/github/shivam-singh-git/Brain-tumor-detection/blob/main/Copy_of_newbtd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

encoder=OneHotEncoder()
encoder.fit([[1], [0]])


X=[]
paths=[]
y=[]
z = []

for r, d, f in os.walk(r"/content/drive/MyDrive/btd/yes"):
  for file in f:
    if '.jpg'in file:
      paths.append(os.path.join(r, file))
for path in paths:
  img=Image.open(path)
  img=img.resize((128, 128))
  img=np.array(img)
  if(img.shape==(128, 128,3)):
    X.append(np.array(img))
    y.append(encoder.transform([[1]]).toarray())


paths=[]
for r, d, f in os.walk(r"/content/drive/MyDrive/btd/no"):
  for file in f:
    if '.jpg'in file :
      paths.append(os.path.join(r, file))
for path in paths:
  img=Image.open(path)
  img= img.resize((128,128))
  img=np.array(img)
  if(img.shape==(128,128,3)):
    X.append(np.array(img))
    y.append(encoder.transform([[0]]).toarray())


X=np.array(X)
y=np.array(y)
y=y.reshape(2900, 2)

paths = []

for r, d, f in os.walk(r"/content/drive/MyDrive/btd/pred"):
  for file in f:
    if '.jpg'in file :
      paths.append(os.path.join(r, file))
for path in paths:
  img=Image.open(path)
  img= img.resize((128,128))
  img=np.array(img)
  if(img.shape==(128,128,3)):
    z.append(np.array(img))



z=np.array(z)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)



model=Sequential()
model.add(Conv2D(32, kernel_size=(2,2), input_shape=(128,128,3), padding='Same'))
model.add(Conv2D(32, kernel_size=(2,2), activation='relu', padding='Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(2,2), activation='relu', padding='Same'))
model.add(Conv2D(64, kernel_size=(2,2), activation='relu', padding='Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss= 'categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])

model.fit(X_train, y_train, batch_size= 128, epochs =10, verbose =0,  validation_data =(X_test, y_test))


loss, accuracy =model.evaluate(X_test, y_test, verbose=0)
print('Model Loss: %0.2f, accuracy: %0.2f' %((loss), (accuracy*100)))


print("Evaluate model on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)


(2320, 128, 128, 3) (580, 128, 128, 3) (2320, 2) (580, 2)
Model Loss: 0.38, accuracy: 87.76
Evaluate model on test data
5/5 [==============================] - 0s 30ms/step - loss: 0.3801 - accuracy: 0.8776
test loss, test acc: [0.3800888657569885, 0.8775861859321594]


In [ ]:
def plot_metrics(history):

    train_loss = history['loss']
    val_loss = history['val_loss']
    train_acc = history['acc']
    val_acc = history['val_acc']

    # Loss
    plt.figure()
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title('Loss')
    plt.legend()
    plt.show()

    # Accuracy
    plt.figure()
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
history = model.history.history

In [ ]:
for keys in history.keys():
  print(keys)

In [ ]:
plot_metrics(history)

KeyError: 'loss'

In [ ]:
img = Image.open(r"/content/drive/MyDrive/btd/pred/pred2.jpg")
img = img.resize((128, 128))

img = np.array(img)

img = np.reshape(img, (1, 128, 128, 3))

prediction = model.predict(img)

print(prediction)

if prediction[0][0] > prediction[0][1]:
    print("No tumor")
else:
    print("yes tumor")
print(prediction)

1/1 [==============================] - 0s 30ms/step
[[0.38940385 0.8296915 ]]
yes tumor
[[0.38940385 0.8296915 ]]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
